In [0]:
import pandas as pd

In [0]:
%sql
/*Formula for WFC*/
/*
FederalFundsSoldSecuritiesPurchasedUnderResaleAgreementsOtherShortTermInvestments (105330000000)
+
CashAndDueFromBanks (37080000000)
+ 
InterestBearingDepositsInBanksAndOtherFinancialInstitutionsNetOfAllowanceForCreditLosses (166281000000)
*/

/*Formula for JPM*/
/*
FederalFundsSoldAndSecuritiesPurchasedUnderAgreementsToResell  (Money Market Investments)
+
CashAndDueFromBanks (Cash)
+ 
InterestBearingDepositsInBanks (Cash And Due from Banks)
*/

/*Formula for BAC*/
/*
FederalFundsSoldAndSecuritiesPurchasedUnderAgreementsToResell (Money Market Investment)
+
CashAndDueFromBanks (Cash)
+
InterestBearingDepositsInBanks (Cash and Due from Banks)
+
TimeDepositsAndOtherShortTermInvestments (Interest Bearing Deposit Assets)
*/

/*Formula for C*/
/*
CarryingValueOfFederalFundsSoldSecuritiesPurchasedUnderAgreementsToResellAndDepositsPaidForSecuritiesBorrowed
+
CashAndDueFromBanks
+
DepositsWithBanksAndOtherFinancialInstitutionsNetOfAllowance
*/

/*Formula for GS*/
/*
CashAndCashEquivalentsAtCarryingValue
+
FederalFundsSoldAndSecuritiesPurchasedUnderAgreementsToResell
*/


/*Formula For MS*/
/*
SecuritiesPurchasedUnderAgreementsToResell (Money Market)
+
CashAndCashEquivalentsAtCarryingValue
*/

In [0]:
companies = ['WFC','JPM','BAC','C','GS','MS']

In [0]:
standard_labels = [
    "FederalFundsSoldSecuritiesPurchasedUnderResaleAgreementsOtherShortTermInvestments", #Money Market
    "FederalFundsSoldAndSecuritiesPurchasedUnderAgreementsToResell", #Money Market
    "CarryingValueOfFederalFundsSoldSecuritiesPurchasedUnderAgreementsToResellAndDepositsPaidForSecuritiesBorrowed",#Money Market
    "SecuritiesPurchasedUnderAgreementsToResell", #Money Market

    "CashAndDueFromBanks", #Cash
    "CashAndCashEquivalentsAtCarryingValue", #Cash

    "InterestBearingDepositsInBanksAndOtherFinancialInstitutionsNetOfAllowanceForCreditLosses", #Cash and Due from Banks
    "InterestBearingDepositsInBanks",#Cash and Due from Banks
    "TimeDepositsAndOtherShortTermInvestments",#Cash and Due from Banks
    "DepositsWithBanksAndOtherFinancialInstitutionsNetOfAllowance",#Cash and Due from Banks

]

standard_labels = str(tuple(standard_labels))

In [0]:
money_market_list = ["FederalFundsSoldSecuritiesPurchasedUnderResaleAgreementsOtherShortTermInvestments", #Money Market
    "FederalFundsSoldAndSecuritiesPurchasedUnderAgreementsToResell", #Money Market
    "CarryingValueOfFederalFundsSoldSecuritiesPurchasedUnderAgreementsToResellAndDepositsPaidForSecuritiesBorrowed",#Money Market
    "SecuritiesPurchasedUnderAgreementsToResell"]

cash_list = [  "CashAndDueFromBanks", #Cash
    "CashAndCashEquivalentsAtCarryingValue"]

cash_and_due_from_banks_list = ["InterestBearingDepositsInBanksAndOtherFinancialInstitutionsNetOfAllowanceForCreditLosses", #Cash and Due from Banks
    "InterestBearingDepositsInBanks",#Cash and Due from Banks
    "DepositsWithBanksAndOtherFinancialInstitutionsNetOfAllowance"]

time_deposits_list = ["TimeDepositsAndOtherShortTermInvestments",""]

money_market = str(tuple(money_market_list))
cash = str(tuple(cash_list))
cash_and_due_from_banks = str(tuple(cash_and_due_from_banks_list))
time_deposits = str(tuple(time_deposits_list))

In [0]:

df_staging = spark.sql(f""" 
with fact as (

select 

fact.standard_label
,case when fact.standard_label in {money_market} then 'money_market'
when fact.standard_label in {cash} then 'cash'
when fact.standard_label in {cash_and_due_from_banks} then 'cash_and_due_from_banks'
when fact.standard_label in {time_deposits} then 'time_deposits'
else 'other' end as standard_label_revised

,dc.company_stock_symbol
,fact.reported_period
,fact.value
from 
operations.finance_staging.fact_staging_financial_statement_tbl fact
left join 
operations.finance.dim_company dc on dc.company_bigint_key = fact.company_bigint_key
where 
financial_statement = 'BS'  and value_segment is null
and 
reported_period = end_reported_period 
and
dc.company_stock_symbol In ('WFC','JPM','BAC','C','GS','MS')
and 
fact.standard_label in {standard_labels}
    
)

select * from fact""")



In [0]:
df = pd.DataFrame(df_staging.toPandas())

In [0]:
pivot_df = df.pivot_table(
    index=["company_stock_symbol", "reported_period"],
    columns="standard_label_revised",
    values="value",
    aggfunc="first"
).reset_index()

pivot_df = pivot_df.fillna(0)

pivot_df['total_cash'] = pivot_df['cash']+pivot_df['money_market']+pivot_df['cash_and_due_from_banks'] + pivot_df['time_deposits'] 


In [0]:
display(pivot_df)